In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

base = "https://www.sports-reference.com"
season = 2024

In [2]:
# link to big-ten for 2024
season = "/cbb/conferences/big-ten/women/2024.html"
page = requests.get(base + season) # ---- request
print(page)
soup = BeautifulSoup(page.content, "html.parser")

table = soup.find(id='big-ten-stats')
teams = pd.read_html(str(table), header=1)[0]

teams['link'] = teams['School'].apply(lambda x: '/cbb/schools/' + x.replace(' ', '-').lower() + '/women/2024-starters.html')
teams['link']

<Response [200]>


0      /cbb/schools/ohio-state/women/2024-starters.html
1         /cbb/schools/indiana/women/2024-starters.html
2            /cbb/schools/iowa/women/2024-starters.html
3     /cbb/schools/michigan-state/women/2024-starter...
4        /cbb/schools/nebraska/women/2024-starters.html
5        /cbb/schools/maryland/women/2024-starters.html
6        /cbb/schools/michigan/women/2024-starters.html
7      /cbb/schools/penn-state/women/2024-starters.html
8        /cbb/schools/illinois/women/2024-starters.html
9       /cbb/schools/wisconsin/women/2024-starters.html
10      /cbb/schools/minnesota/women/2024-starters.html
11         /cbb/schools/purdue/women/2024-starters.html
12    /cbb/schools/northwestern/women/2024-starters....
13        /cbb/schools/rutgers/women/2024-starters.html
Name: link, dtype: object

In [35]:
game_dfs = []
for i in range(0, teams.shape[0]):
#for i in range(0, 1): # just Ohio State

    # use .iloc to grab last column for each row [i, -1]
    team_page = requests.get(base + teams.iloc[i, -1]) # ---- request
    print(base + teams.iloc[i, -1])

    # create soup element
    soup_games = BeautifulSoup(team_page.content, "html.parser")

    # get table for starters
    games = soup_games.find(id='starters')
    
    game_links = []
    for row in games.tbody.find_all('tr'):

        # Find all data for each column
        game_data = row.find_all('td')

        game_links.append(game_data[2].find('a', href=True).get('href'))


    for game_link in game_links: 
        print('Starting: ', game_link)
        box_score_page = requests.get(base + game_link) # ---- request

        bsp = BeautifulSoup(box_score_page.content, "html.parser")

        boxes = bsp.find(id='boxes')

        tbls = boxes.find_all('table')

        # get game_id
        game_id = game_link[15:-7]

        # get t1 box score data
        t1 = pd.read_html(str(tbls[0]), header=1)[0]
        t1_team = tbls[0].get('id')

        # get team name
        t1['team'] = t1_team[16:-2]
        t1['game_id'] = game_id

        # get t2 box score data
        t2 = pd.read_html(str(tbls[2]), header=1)[0]
        t2_team = tbls[2].get('id')

        # get team name
        t2['team'] = t2_team[16:-2]
        t2['game_id'] = game_id

        # concat all box scores for game
        game_dfs.append(t1)
        game_dfs.append(t2)

        print('Done - current games in db: ', int(len(game_dfs) / 2))
        time.sleep(4) # sleep at least 3 seconds for 20 requests per min

all_games = pd.concat(game_dfs, axis=0)
all_games = all_games[(all_games['Starters'] != 'Reserves') & (all_games['Starters'] != 'School Totals')]
all_games[all_games.columns[1:-2]] = all_games[all_games.columns[1:-2]].apply(pd.to_numeric)
all_games.head()

https://www.sports-reference.com/cbb/schools/ohio-state/women/2024-starters.html
Starting:  /cbb/boxscores/2023-11-06-14-ohio-state_w.html
Done - current games in db:  1
Starting:  /cbb/boxscores/2023-11-12-13-ohio-state_w.html
Done - current games in db:  2
Starting:  /cbb/boxscores/2023-11-16-19-ohio-state_w.html
Done - current games in db:  3
Starting:  /cbb/boxscores/2023-11-20-13-east-carolina_w.html
Done - current games in db:  4
Starting:  /cbb/boxscores/2023-11-22-18-ohio-state_w.html
Done - current games in db:  5
Starting:  /cbb/boxscores/2023-11-26-13-ohio-state_w.html
Done - current games in db:  6
Starting:  /cbb/boxscores/2023-12-03-17-tennessee_w.html
Done - current games in db:  7
Starting:  /cbb/boxscores/2023-12-05-19-ohio-state_w.html
Done - current games in db:  8
Starting:  /cbb/boxscores/2023-12-10-13-ohio-state_w.html
Done - current games in db:  9
Starting:  /cbb/boxscores/2023-12-15-18-ohio-state_w.html
Done - current games in db:  10
Starting:  /cbb/boxscores/

,Starters,MP,FG,FGA,FG%,2P,2PA,2P%,3P,3PA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,team,game_id
0,Jacy Sheldon,37,10,19,0.526,8,14,0.571,2,5,...,2,4,2,5,0,4,2,28,ohio-state,2023-11-06-14-ohio-state
1,Celeste Taylor,35,1,8,0.125,0,3,0.000,1,5,...,3,3,2,2,0,2,5,7,ohio-state,2023-11-06-14-ohio-state
2,Cotie McMahon,34,3,11,0.273,2,8,0.250,1,3,...,4,5,2,1,0,5,4,7,ohio-state,2023-11-06-14-ohio-state
3,Taylor Thierry,33,5,6,0.833,5,5,1.000,0,1,...,6,8,0,2,0,2,3,16,ohio-state,2023-11-06-14-ohio-state
4,Rebeka Mikulasikova,21,2,5,0.400,2,3,0.667,0,2,...,3,4,1,0,0,1,1,8,ohio-state,2023-11-06-14-ohio-state


In [36]:
all_games.duplicated(subset=['Starters', 'team', 'game_id'])

0    False
1    False
2    False
3    False
4    False
     ...  
2     True
3     True
4     True
6     True
7     True
Length: 8174, dtype: bool

In [39]:
games_dedup = all_games.drop_duplicates(subset=['Starters', 'team', 'game_id'], keep='first')
games_dedup.shape[0]

5701

In [40]:
games_dedup.to_csv('2023_box_scores.csv')

In [41]:
games_dedup.describe()

,MP,FG,FGA,FG%,2P,2PA,2P%,3P,3PA,3P%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
count,5701.000000,5701.000000,5701.000000,5091.000000,5701.000000,5701.000000,4622.000000,5701.000000,5701.000000,3581.000000,...,2728.000000,5701.000000,5701.000000,5701.000000,5701.000000,5701.000000,5701.000000,5701.000000,5701.000000,5701.000000
mean,19.791616,2.578320,5.971584,0.406179,1.891949,3.919839,0.459244,0.686371,2.051745,0.297367,...,0.719358,0.876688,2.281003,3.157692,1.501316,0.746185,0.305911,1.491317,1.676022,7.054552
std,11.041615,2.543319,4.808206,0.267752,2.169220,3.745884,0.314224,1.167636,2.547091,0.317437,...,0.306062,1.253928,2.327405,2.987437,1.909865,1.045267,0.684476,1.563668,1.422292,6.784865
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,11.000000,0.000000,2.000000,0.250000,0.000000,1.000000,0.250000,0.000000,0.000000,0.000000,...,0.500000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
50%,20.000000,2.000000,5.000000,0.400000,1.000000,3.000000,0.500000,0.000000,1.000000,0.250000,...,0.750000,0.000000,2.000000,2.000000,1.000000,0.000000,0.000000,1.000000,1.000000,6.000000
75%,29.000000,4.000000,9.000000,0.563000,3.000000,6.000000,0.667000,1.000000,3.000000,0.500000,...,1.000000,1.000000,3.000000,5.000000,2.000000,1.000000,0.000000,2.000000,3.000000,11.000000
max,50.000000,17.000000,34.000000,1.000000,17.000000,23.000000,1.000000,9.000000,20.000000,1.000000,...,1.000000,12.000000,16.000000,21.000000,15.000000,8.000000,7.000000,13.000000,6.000000,49.000000


In [42]:
games_dedup[games_dedup['3PA'] == max(games_dedup['3PA'])]

,Starters,MP,FG,FGA,FG%,2P,2PA,2P%,3P,3PA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,team,game_id
0,Caitlin Clark,37,14,34,0.412,6,14,0.429,8,20,...,1,1,5,3,1,6,2,40,iowa,2024-01-02-21-iowa
